In [12]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope for the APIs
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly", 
         "https://www.googleapis.com/auth/drive.readonly"]

# Provide the path to the credentials JSON file you downloaded
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/alexandercappelen/Documents/keys/frb-elite-88e4dcc7ec5c.json', scope)

# Authorize and create the client
client = gspread.authorize(creds)

# Open the sheet by name
player_data = client.open("frb-volley-game-stats").worksheet("player-offense")

# Fetch all records (rows) from the sheet
player_data = player_data.get_all_records()

# Print the data or convert it into a DataFrame
import pandas as pd
df = pd.DataFrame(player_data)
df

,date,match-id,match,set,player,serves,aces,serve-error,points-won,point-scoring-pct,attack,kills,attack-errors,kill-pct,kill-effic
0,08/12/2024,141108,frb-gentofte,1,Lasse,2,1,0,1,50.0,1,1,0,100.000000,1.000000
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0,0,0,0.0,0,0,0,0.000000,0.000000
2,08/12/2024,141108,frb-gentofte,1,Kristian,5,0,2,3,60.0,2,1,0,50.000000,0.500000
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0,0,0,0.0,0,0,0,0.000000,0.000000
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0,0,0,0.0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,23/11/2024,141099,frb-hvidovre,5,Mads,2,0,0,1,50.0,0,0,0,0.000000,0.000000
552,23/11/2024,141099,frb-hvidovre,5,Martin,0,0,0,0,0.0,0,0,0,0.000000,0.000000
553,23/11/2024,141099,frb-hvidovre,5,Mikkel,0,0,0,0,0.0,3,2,0,66.666667,0.666667
554,23/11/2024,141099,frb-hvidovre,5,Frederik,0,0,0,0,0.0,0,0,0,0.000000,0.000000


In [13]:
df["error-pct"] = df["attack-errors"] / df["attack"]

summary = df.groupby(['match', 'player']).agg(
    attack_attempts=('attack', 'sum'),
    total_kills=('kills', 'sum'),
    attack_errors=('attack-errors', 'sum')
).reset_index()

summary["kill_pct"] = ((summary["total_kills"]) / summary["attack_attempts"] * 100).round(0)
summary["error_pct"] = ((summary["attack_errors"]) / summary["attack_attempts"] * 100).round(0)
summary["kill_effic"] = ((summary["total_kills"] - summary["attack_errors"]) / summary["attack_attempts"])#.round(3)
summary = summary.dropna()
summary["kill_pct"] = summary["kill_pct"].astype(int)
summary["error_pct"] = summary["error_pct"].astype(int)
summary["kill_effic"] = summary["kill_effic"].round(3)

In [14]:
total_summary = df.groupby('player').agg(
    attack_attempts=('attack', 'sum'),
    total_kills=('kills', 'sum'),
    attack_errors=('attack-errors', 'sum')
).reset_index()
total_summary

total_summary["kill_pct"] = ((total_summary["total_kills"]) / total_summary["attack_attempts"] * 100).round(0)
total_summary["error_pct"] = ((total_summary["attack_errors"]) / total_summary["attack_attempts"] * 100).round(0)
total_summary["kill_effic"] = ((total_summary["total_kills"] - total_summary["attack_errors"]) / total_summary["attack_attempts"])#.round(3)
total_summary = total_summary.dropna()
total_summary["kill_pct"] = total_summary["kill_pct"].astype(int)
total_summary["error_pct"] = total_summary["error_pct"].astype(int)
total_summary["kill_effic"] = total_summary["kill_effic"].round(3)
total_summary['match'] = "all-matches"
total_summary

,player,attack_attempts,total_kills,attack_errors,kill_pct,error_pct,kill_effic,match
1,Ando,7,6,0,86,0,0.857,all-matches
2,Bo,37,14,7,38,19,0.189,all-matches
3,Boerme,118,46,20,39,17,0.220,all-matches
4,Bosse,12,2,3,17,25,-0.083,all-matches
5,Frederik,4,1,1,25,25,0.000,all-matches
6,Gustav,65,24,11,37,17,0.200,all-matches
7,Kristian,51,24,5,47,10,0.373,all-matches
8,Lasse,41,17,3,41,7,0.341,all-matches
9,Mads,6,1,0,17,0,0.167,all-matches
10,Martin,145,62,29,43,20,0.228,all-matches


In [15]:
import json

summary = pd.concat([summary, total_summary], ignore_index=True)

# Convert DataFrame to JSON
data = summary.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../player-offense-per-game.json", "w") as f:
    json.dump(data, f, indent=4)

In [16]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope for the APIs
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly", 
         "https://www.googleapis.com/auth/drive.readonly"]

# Provide the path to the credentials JSON file you downloaded
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/alexandercappelen/Documents/keys/frb-elite-88e4dcc7ec5c.json', scope)

# Authorize and create the client
client = gspread.authorize(creds)

# Open the sheet by name
player_defense = client.open("frb-volley-game-stats").worksheet("player-defense")
player_positions = client.open("frb-volley-game-stats").worksheet("player-positions")
player_reception = client.open("frb-volley-game-stats").worksheet("player-reception")

# Fetch all records (rows) from the sheet
player_defense_data = player_defense.get_all_records()
player_reception_data = player_reception.get_all_records()

# Print the data or convert it into a DataFrame
import pandas as pd
player_positions_df = pd.DataFrame(player_positions.get_all_records())
df = pd.DataFrame(player_defense_data)
df['pass-rating'] = pd.to_numeric(df['pass-rating'], errors='coerce')
df['pass-attempt'] = pd.to_numeric(df['pass-attempt'])
df['pass-attempt'] = df['pass-attempt'].fillna(0).astype(int)

df['pass-error'] = pd.to_numeric(df['pass-error'])
df['pass-error'] = df['pass-error'].fillna(0).astype(int)

df['digs'] = pd.to_numeric(df['digs'])
df['digs'] = df['digs'].fillna(0).astype(int)

df['dig-error'] = pd.to_numeric(df['dig-error'])
df['dig-error'] = df['dig-error'].fillna(0).astype(int)

df['blocks'] = pd.to_numeric(df['blocks'])
df['blocks'] = df['blocks'].fillna(0).astype(int)

df['block-error'] = pd.to_numeric(df['block-error'])
df['block-error'] = df['block-error'].fillna(0).astype(int)


#df["error-pct"] = df["attack-errors"] / df["attack"]
#df["pass-rating"].unique()
df


,date,match-id,match,set,player,pass-attempt,pass-error,pass-rating,digs,dig-error,blocks,block-error,defense-errors
0,08/12/2024,141108,frb-gentofte,1,Lasse,0,0,NaN,1,1,0,2,3
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0,NaN,0,0,0,0,0
2,08/12/2024,141108,frb-gentofte,1,Kristian,0,0,NaN,0,0,0,0,0
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0,NaN,0,0,0,0,0
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
551,23/11/2024,141099,frb-hvidovre,5,Mads,0,0,NaN,0,0,0,0,
552,23/11/2024,141099,frb-hvidovre,5,Martin,0,0,NaN,0,0,0,0,
553,23/11/2024,141099,frb-hvidovre,5,Mikkel,0,0,NaN,0,0,0,0,
554,23/11/2024,141099,frb-hvidovre,5,Frederik,0,0,NaN,0,0,0,0,


In [17]:
# Passing statistics per game

df_player_reception = pd.DataFrame(player_reception_data)

# Group by player to get player-level stats
player_reception_stats_total = df_player_reception.groupby('player').agg(
    avg_pass_rating=('pass-rating', 'mean'),
    number_pass_attempts=('pass-attempt', 'sum'),
    positive_pct=('pass-2', 'mean')
).reset_index()

player_reception_stats_total["match"] = "all-matches"

In [18]:
# Group by player to get player-level stats
player_reception_stats = df_player_reception.groupby(['match','player']).agg(
    avg_pass_rating=('pass-rating', 'mean'),
    number_pass_attempts=('pass-attempt', 'sum'),
    positive_pct=('pass-2', 'mean')
).reset_index()

player_reception_stats

,match,player,avg_pass_rating,number_pass_attempts,positive_pct
0,avk-frb,Alex,2.027778,34,2.4
1,avk-frb,Ando,0.000000,0,0.0
2,avk-frb,Bo,0.000000,0,0.0
3,avk-frb,Boerme,2.206667,22,1.6
4,avk-frb,Bosse,0.000000,0,0.0
...,...,...,...,...,...
131,gentofte-frb,Mikkel,0.000000,0,0.0
132,gentofte-frb,Nico,0.000000,0,0.0
133,gentofte-frb,Nicola,0.681818,15,1.2
134,gentofte-frb,Soeren,0.000000,0,0.0


In [19]:
rating_summary = df_player_reception[df_player_reception["pass-attempt"] > 0]  #[["player", "pass-rating"]]


rating_summary = rating_summary.groupby(["match", "player"]).agg(
    average_pass_rating=('pass-rating', 'mean')
).reset_index()
rating_summary

summary = df_player_reception.groupby(["match", "player"]).sum().reset_index()



# Calculate positive and perfect pass counts
summary['positive_passes'] = summary['pass-2'] + summary['pass-3']
summary['perfect_passes'] = summary['pass-3']

# Calculate percentages
summary['positive_percentage'] = (summary['positive_passes'] / summary['pass-attempt']) * 100
summary['perfect_percentage'] = (summary['perfect_passes'] / summary['pass-attempt']) * 100
summary['error_percentage'] = (summary['pass-error'] / summary['pass-attempt']) * 100
summary = summary[["match", 'player', 'pass-attempt', 'positive_percentage', 'perfect_percentage', "error_percentage"]]


summary[summary["player"] == "Bosse"]


,match,player,pass-attempt,positive_percentage,perfect_percentage,error_percentage
4,avk-frb,Bosse,0,NaN,NaN,NaN
18,frb-dtu,Bosse,0,NaN,NaN,NaN
32,frb-gentofte,Bosse,0,NaN,NaN,NaN
46,frb-holte,Bosse,8,50.0,37.5,0.0
62,frb-hvidovre,Bosse,10,50.0,10.0,20.0
78,frb-koge,Bosse,0,NaN,NaN,NaN
94,frb-kv61,Bosse,0,NaN,NaN,NaN
110,frb-vli,Bosse,0,NaN,NaN,NaN
124,gentofte-frb,Bosse,1,100.0,0.0,0.0


In [20]:


final_summary_per_game = pd.merge(
    summary[["match",'player', 'pass-attempt', 'positive_percentage', 'perfect_percentage', 'error_percentage']],
    rating_summary[["match","player","average_pass_rating"]],
    on=["match", "player"],
    how="inner"
)

final_summary_per_game[final_summary_per_game["player"] == "Bosse"]

,match,player,pass-attempt,positive_percentage,perfect_percentage,error_percentage,average_pass_rating
19,frb-holte,Bosse,8,50.0,37.5,0.0,1.583333
24,frb-hvidovre,Bosse,10,50.0,10.0,20.0,1.437500
47,gentofte-frb,Bosse,1,100.0,0.0,0.0,2.000000


In [21]:
# Seperat df for "pass rating", der ikke skal summeres, men i stedet bruge gennemsnit for den enkelte spiller for hver kampe
rating_summary = df_player_reception[df_player_reception["pass-attempt"] > 0]  #[["player", "pass-rating"]]
rating_summary = rating_summary[["player", "pass-rating"]]

rating_summary = rating_summary.groupby(["player"]).agg(
    average_pass_rating=('pass-rating', 'mean')
).reset_index()

# df der summerer modtagningstallene for den enkelte spiller for hver kampe
summary = df_player_reception[df_player_reception["pass-attempt"] > 0]
summary = summary[["player","pass-attempt", "pass-error", "pass-1", "pass-2", "pass-3", "overpass-in-play"]].groupby(["player"]).sum().reset_index()


# Calculate positive and perfect pass counts
summary['positive_passes'] = summary['pass-2'] + summary['pass-3']
summary['perfect_passes'] = summary['pass-3']

# Calculate percentages
summary['positive_percentage'] = (summary['positive_passes'] / summary['pass-attempt']) * 100
summary['perfect_percentage'] = (summary['perfect_passes'] / summary['pass-attempt']) * 100
summary['error_percentage'] = (summary['pass-error'] / summary['pass-attempt']) * 100
summary = summary[['player', 'pass-attempt', 'positive_percentage', 'perfect_percentage', "error_percentage"]]

final_summary = pd.merge(
    summary[['player', 'pass-attempt', 'positive_percentage', 'perfect_percentage', 'error_percentage']],
    rating_summary[["player","average_pass_rating"]],
    on="player"
)

final_summary["match"] = "all-matches"
final_summary = final_summary[["match", "player", "pass-attempt", "positive_percentage", "perfect_percentage", "error_percentage", "average_pass_rating"]]
final_summary

combined_pass_summary = pd.concat([final_summary, final_summary_per_game], ignore_index=True)

combined_pass_summary = combined_pass_summary[combined_pass_summary['positive_percentage'].notna()]

combined_pass_summary[combined_pass_summary["player"] == "Bosse"]

,match,player,pass-attempt,positive_percentage,perfect_percentage,error_percentage,average_pass_rating
3,all-matches,Bosse,19,52.631579,21.052632,10.526316,1.608333
30,frb-holte,Bosse,8,50.000000,37.500000,0.000000,1.583333
35,frb-hvidovre,Bosse,10,50.000000,10.000000,20.000000,1.437500
58,gentofte-frb,Bosse,1,100.000000,0.000000,0.000000,2.000000


In [22]:
# Convert DataFrame to JSON
data = combined_pass_summary.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../player-passing-per-game.json", "w") as f:
    json.dump(data, f, indent=4)